In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import itertools

In [10]:
with open('eleuther_eval_jobs.tsv', 'r') as f:
    df1 = pd.read_csv(f, sep='\t', index_col=None)

df1['eval_method'] = 'eleuther'
with open('composer_eval_jobs.tsv', 'r') as f:
    df2 = pd.read_csv(f, sep='\t', index_col=None)

df2['eval_method'] = 'composer'

df = pd.concat([df1, df2])
df

,gpu_num,parameters-icl_tasks,parameters-model,run_name,run_time,accuracy,eval_method
0,64,piqa,125m,gpt-eval-gpu-num-64-icl-tasks-piqa-model-125m-...,718.886351,0.628945,eleuther
1,64,piqa,1.3b,gpt-eval-gpu-num-64-icl-tasks-piqa-model-1-3b-...,1927.111743,0.707835,eleuther
2,64,piqa,2.7b,gpt-eval-gpu-num-64-icl-tasks-piqa-model-2-7b-...,3260.289173,0.731774,eleuther
3,32,piqa,125m,gpt-eval-gpu-num-32-icl-tasks-piqa-model-125m-...,733.135527,0.628945,eleuther
4,32,piqa,1.3b,gpt-eval-gpu-num-32-icl-tasks-piqa-model-1-3b-...,2025.282325,0.707835,eleuther
...,...,...,...,...,...,...,...
61,8,lambada,1.3b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-1-3...,69.163373,0.571705,composer
62,8,lambada,2.7b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-2-7...,117.466429,0.622093,composer
63,8,piqa,125m,gpt-eval-gpu-num-8-icl-tasks-piqa-model-125m-5523,123.490922,0.623217,composer
64,8,piqa,1.3b,gpt-eval-gpu-num-8-icl-tasks-piqa-model-1-3b-3502,476.884176,0.709726,composer


In [34]:
df = df.dropna()
graph_partition_cols = ['parameters-icl_tasks', 'parameters-model'] # diff values will be on diff graphs
line_partition_cols = ['eval_method'] # diff values will be diff lines

graph_partition_vals = {
    k: set(df[k]) for k in graph_partition_cols
}
line_partition_vals = {
    k: set(df[k]) for k in line_partition_cols
}
x_col = 'gpu_num'
y_col = 'run_time'
df.sort_values(by=['gpu_num'], inplace=True)
df

,gpu_num,parameters-icl_tasks,parameters-model,run_name,run_time,accuracy,eval_method
25,1,lambada,1.3b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-1-3...,546.160601,0.571900,composer
29,1,piqa,2.7b,gpt-eval-gpu-num-8-icl-tasks-piqa-model-2-7b-0714,6493.076296,0.728205,composer
28,1,piqa,1.3b,gpt-eval-gpu-num-8-icl-tasks-piqa-model-1-3b-1154,3804.749575,0.707704,composer
27,1,piqa,125m,gpt-eval-gpu-num-8-icl-tasks-piqa-model-125m-5866,979.931637,0.623196,composer
26,1,lambada,2.7b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-2-7...,931.951627,0.622162,composer
...,...,...,...,...,...,...,...
2,64,piqa,2.7b,gpt-eval-gpu-num-64-icl-tasks-piqa-model-2-7b-...,3260.289173,0.731774,eleuther
1,64,piqa,1.3b,gpt-eval-gpu-num-64-icl-tasks-piqa-model-1-3b-...,1927.111743,0.707835,eleuther
12,64,lambada,125m,gpt-eval-gpu-num-64-icl-tasks-lambada-model-12...,1017.287463,0.373569,eleuther
16,64,lambada,1.3b,gpt-eval-gpu-num-64-icl-tasks-lambada-model-1-...,2695.111596,0.572094,eleuther


In [35]:
def partition_graphs(df, graph_partition_vals):
    keys, vals = graph_partition_vals.keys(), graph_partition_vals.values()
    
    dataset_graph_partitions = []
    for combo in itertools.product(*vals):
        filter_string = ""
        
        df_part = {}
        df_part['choices'] = {}
        
        for col_name, col_val in zip(keys, combo):
            filter_string += f"`{col_name}` == '{col_val}' and"
            df_part['choices'][col_name] = col_val
            
        filter_string = filter_string[:-4]
        df_subset = df.query(filter_string)
        df_part['df'] = df_subset
        dataset_graph_partitions.append(df_part)
    
    return dataset_graph_partitions



graph_partitions = partition_graphs(df, graph_partition_vals)

In [20]:
def partition_lines(df, line_partition_vals):
    keys, vals = line_partition_vals.keys(), line_partition_vals.values()
    dataset_line_partitions = []
    for combo in itertools.product(*vals):
        filter_string = ""
        
        df_part = {}
        df_part['choices'] = {}
        
        for col_name, col_val in zip(keys, combo):
            filter_string += f"`{col_name}` == '{col_val}' and"
            df_part['choices'][col_name] = col_val
            
        filter_string = filter_string[:-4]
        df_subset = df.query(filter_string)
        df_part['df'] = df_subset
        dataset_line_partitions.append(df_part)
    return dataset_line_partitions 

def make_title(choices):
    if 'eval_method' in choices:
        return f"{choices['eval_method']}-based eval, {choices['parameters-icl_tasks'].upper()}"
    elif 'parameters-model' in choices:
        return f"{choices['parameters-model']} parameter model, {choices['parameters-icl_tasks'].upper()}"
    else:
        return None
   



In [36]:


for idx, graph_info in enumerate(graph_partitions):
    choices = graph_info['choices']
    line_partitions = partition_lines(graph_info['df'], line_partition_vals)
    legend = []
    for line in line_partitions:
        legend.append(', '.join(line['choices'].values()))
        plt.plot(line['df'][x_col],line['df'][y_col])
    
    
    plt.legend(legend)
    plt.xscale("log")
    plt.yscale("log")

    
    plt.title(make_title(choices))
    plt.xlabel('Number of GPUs')
    plt.ylabel('Run time (seconds)')
    plt.savefig(f"fig{idx}.png", )
    plt.clf()



<Figure size 640x480 with 0 Axes>

In [27]:
line['df']

,gpu_num,parameters-icl_tasks,parameters-model,run_name,run_time,accuracy,eval_method
26,1,lambada,2.7b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-2-7...,931.951627,0.622162,composer
13,8,lambada,2.7b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-2-7...,117.373195,0.623253,composer
62,8,lambada,2.7b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-2-7...,117.466429,0.622093,composer
56,16,lambada,2.7b,gpt-eval-gpu-num-16-icl-tasks-lambada-model-2-...,59.242780,0.621904,composer
9,16,lambada,2.7b,gpt-eval-gpu-num-16-icl-tasks-lambada-model-2-...,59.174347,0.623253,composer
50,32,lambada,2.7b,gpt-eval-gpu-num-32-icl-tasks-lambada-model-2-...,30.257925,0.622106,composer
5,32,lambada,2.7b,gpt-eval-gpu-num-32-icl-tasks-lambada-model-2-...,30.006946,0.623253,composer
1,64,lambada,2.7b,gpt-eval-gpu-num-64-icl-tasks-lambada-model-2-...,15.647013,0.622106,composer
44,64,lambada,2.7b,gpt-eval-gpu-num-64-icl-tasks-lambada-model-2-...,15.661913,0.622106,composer


In [ ]:
x,y = zip(*big_models)

In [ ]:
big_models = [(7e10,110.333),
(2.02e11,340.168),
(3.23e11,544.74),
(4.43e11,741.209)]


def billions_formatter(x, pos):
    return f'{x / 1000000000}'

In [ ]:

fig, ax = plt.subplots()
ax.plot(x,y)
ax.xaxis.set_major_formatter(FuncFormatter(billions_formatter))
plt.title("LAMBADA eval time on 64 GPUs")
plt.xlabel('Number of Parameters (billions)')
plt.ylabel('Run time (seconds)')
plt.savefig(f"fig4.png", )
plt.clf()